# processing

> Necessary scripts to read orbits from different formats

In [ ]:
#| default_exp processing

In [ ]:
#| export
from scipy.interpolate import interp1d
import numpy as np
import time

In [ ]:
from orbit_generation_testing.reader import get_example_orbit_data

In [ ]:
orbit_data= get_example_orbit_data()
orbit_data.shape

In [ ]:
#| export
def resample_3d_array(data, axis, target_size):
    """
    Resample a 3D numpy array along a specified axis using linear interpolation, with correct dimension handling.

    Parameters:
        data (numpy.ndarray): The original 3D array.
        axis (int): The axis along which to perform the interpolation (0, 1, or 2).
        target_size (int): The desired size of the specified axis after interpolation.
    
    Returns:
        numpy.ndarray: A new array with the specified axis resized to `target_size`.
    """
    # Validate the axis
    if axis not in [0, 1, 2]:
        raise ValueError("Invalid axis. Axis must be 0, 1, or 2.")

    # Prepare indices for interpolation
    old_indices = np.linspace(0, 1, num=data.shape[axis])
    new_indices = np.linspace(0, 1, num=target_size)

    # Initialize the output array
    new_shape = list(data.shape)
    new_shape[axis] = target_size
    new_data = np.empty(new_shape, dtype=data.dtype)
    
    # Perform interpolation along the specified axis
    if axis == 0:
        for i in range(data.shape[1]):
            for j in range(data.shape[2]):
                interpolator = interp1d(old_indices, data[:, i, j], kind='linear')
                new_data[:, i, j] = interpolator(new_indices)
    elif axis == 1:
        for i in range(data.shape[0]):
            for j in range(data.shape[2]):
                interpolator = interp1d(old_indices, data[i, :, j], kind='linear')
                new_data[i, :, j] = interpolator(new_indices)
    else:
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                interpolator = interp1d(old_indices, data[i, j, :], kind='linear')
                new_data[i, j, :] = interpolator(new_indices)

    return new_data

In [ ]:
#| export
def estimate_resampling_duration(data, axis, target_size, num_samples=10):
    """
    Estimate the duration of the resampling process for a 3D numpy array based on a sample.

    Parameters:
        data (numpy.ndarray): The original 3D numpy array.
        axis (int): The axis along which to perform the interpolation.
        target_size (int): The desired size of the specified axis after interpolation.
        num_samples (int): The number of random samples to test for estimating duration.

    Returns:
        float: Estimated total duration to process the entire dataset.
    """
    total_elements = data.shape[0]
    sample_indices = np.random.choice(total_elements, num_samples, replace=False)
    sample_data = data[sample_indices]
    
    start_time = time.time()
    # Process each sample
    for sample in sample_data:
        reshaped_sample = sample.reshape(1, *sample.shape)
        resampled_sample = resample_3d_array(reshaped_sample, axis, target_size)
    elapsed_time = time.time() - start_time

    # Calculate average time per sample
    avg_time_per_sample = elapsed_time / num_samples
    # Estimate total time for the entire dataset
    estimated_total_time = avg_time_per_sample * total_elements
    
    return estimated_total_time

In [ ]:
#| export
def average_downsample_3d_array(data, axis, target_size):
    """
    Downsample a 3D numpy array along a specified axis using averaging.

    Parameters:
        data (numpy.ndarray): The original 3D array.
        axis (int): The axis along which to perform the downsampling (0, 1, or 2).
        target_size (int): The desired size of the specified axis after downsampling.
    
    Returns:
        numpy.ndarray: A new array with the specified axis resized to `target_size`.
    """
    # Validate the axis
    if axis not in [0, 1, 2]:
        raise ValueError("Invalid axis. Axis must be 0, 1, or 2.")

    # Determine the number of elements in each block to be averaged
    original_size = data.shape[axis]
    block_size = original_size / target_size

    # Initialize the output array
    new_shape = list(data.shape)
    new_shape[axis] = target_size
    new_data = np.empty(new_shape, dtype=data.dtype)

    # Perform averaging along the specified axis
    if axis == 0:
        for i in range(target_size):
            start_idx = int(i * block_size)
            end_idx = int((i + 1) * block_size)
            new_data[i, :, :] = np.mean(data[start_idx:end_idx, :, :], axis=0)
    elif axis == 1:
        for i in range(target_size):
            start_idx = int(i * block_size)
            end_idx = int((i + 1) * block_size)
            new_data[:, i, :] = np.mean(data[:, start_idx:end_idx, :], axis=1)
    else:  # axis == 2
        for i in range(target_size):
            start_idx = int(i * block_size)
            end_idx = int((i + 1) * block_size)
            new_data[:, :, i] = np.mean(data[:, :, start_idx:end_idx], axis=2)

    return new_data

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)